<a href="https://colab.research.google.com/github/readall/indiatranslator/blob/main/indicTranslate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install tqdm boto3 requests regex sentencepiece sacremoses transformers easyocr
pip install mosestokenizer indic-nlp-library mock sacrebleu tensorboardX pyarrow subword-nmt
git clone https://github.com/pytorch/fairseq.git
cd fairseq
pip install --editable ./
%cd ..
wget https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
unzip indic-en.zip


Process is terminated.


In [15]:
# downloading the indic-indic model
!cd ..
!wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
!unzip m2m.zip

# %cd indicTrans

--2021-08-20 06:28:53--  https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.15.128, 173.194.76.128, 66.102.1.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.15.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4081990185 (3.8G) [application/zip]
Saving to: ‘m2m.zip’

m2m.zip             100%[===================>]   3.80G  41.2MB/s    in 73s     

2021-08-20 06:30:06 (53.6 MB/s) - ‘m2m.zip’ saved [4081990185/4081990185]

Archive:  m2m.zip
   creating: m2m/
   creating: m2m/vocab/
  inflating: m2m/vocab/vocab.SRC     
  inflating: m2m/vocab/vocab.TGT     
  inflating: m2m/vocab/bpe_codes.32k.SRC_TGT  
   creating: m2m/final_bin/
  inflating: m2m/final_bin/dict.TGT.txt  
  inflating: m2m/final_bin/dict.SRC.txt  
   creating: m2m/model/
  inflating: m2m/model/checkpoint_best.pt  


In [15]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 498, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 498 (delta 160), reused 132 (delta 117), pack-reused 297
Receiving objects: 100% (498/498), 1.49 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (287/287), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 3.21 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 23.2

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
import easyocr

In [3]:
reader = easyocr.Reader(['kn','en']) # need to run only once to load model into memory

In [4]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [5]:
image_root_path = "/content/drive/MyDrive/Colab Notebooks/"
kannada_img_file = image_root_path + 'kannada_01.jpg'
kannada_img_file_1 = '/content/drive/MyDrive/Colab Notebooks/kannada_01.jpg'

In [5]:
result = reader.readtext('kannada_02.jpg',detail = 0)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [6]:
result

['|.ಹೊಂದಿಸಿ ಬರೆಯಿರಿ:',
 '೧.ಕಾಪಾಡು',
 'ಹನಿ',
 '೨',
 'ಬುದ್ದಿವಂತ',
 'ಸಂರಕ್ಷಿಸು',
 '೩.ಮಳೆ',
 'ಆಸನ',
 '೪',
 'ಬೆಟ್ಟ',
 'ರಾಮಕೃಷ್ಣ',
 '೫.ಪೀಠ',
 'ಗುಡ್ಡ',
 'Ilಆವರಣದಲ್ಲಿ ಕೊಟ್ಟಿರುವ ಸರಿಯಾದ ಉತ್ತರವನ್ನು ಆರಿಸಿ ಬಿಟ್ಟ ಸ್ಥಳದಲ್ಲಿ ಬರೆಯಿರಿ:',
 '(ಕುವೆಂಪು, ಅಪ್ಪುಗೆ, ಪಂಡಿತರು,',
 'ತ,ಮಳೆ',
 '೧',
 '.ಕೃಷ್ಣದೇವರಾಯನ ಆಸ್ಥಾನದಲ್ಲಿ',
 'ಇದ್ದರು.',
 '೨.ನೀರಿನ ಮೂಲ',
 'ಸಂಸ್ಕೃ ತ']

In [6]:
result = reader.readtext(kannada_img_file, detail = 0)
with open('ka.txt', 'w') as f:
    f.write(str(result))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [7]:
from transformers import AutoTokenizer, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert", keep_accents=True)

model = AutoModel.from_pretrained("ai4bharat/indic-bert")

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [16]:
!bash ./indicTrans/joint_translate.sh ka.txt en_outputs.txt 'kn' 'en' './indic-en'

Fri Aug 20 05:40:52 UTC 2021
Applying normalization and script conversion
python3: can't open file 'scripts/preprocess_translate.py': [Errno 2] No such file or directory
Number of sentences in input: 
Applying BPE
./indicTrans/joint_translate.sh: line 27: en_outputs.txt.norm: No such file or directory
python3: can't open file 'scripts/add_tags_translate.py': [Errno 2] No such file or directory
Decoding
Extracting translations, script conversion and detokenization
python3: can't open file 'scripts/postprocess_translate.py': [Errno 2] No such file or directory
Translation completed


In [17]:
# we need to be in indicTrans directory for translation model to load
%cd indicTrans/


[Errno 2] No such file or directory: 'indicTrans/'
/content/indicTrans


In [21]:
!bash joint_translate.sh /content/ka.txt en_outputs.txt 'kn' 'en' '../indic-en'

Fri Aug 20 05:45:57 UTC 2021
Applying normalization and script conversion
100% 1/1 [00:00<00:00, 29.36it/s]
Number of sentences in input: 1
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [22]:
!cat en_outputs.txt

Set to: ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', '


In [10]:
from indicTrans.inference.engine import Model

indic2en_model = Model(expdir='../indic-en')

Initializing vocab and bpe
Initializing model for translation


In [11]:
indic2en_model.batch_translate(result, 'kn', 'en')

100%|██████████| 21/21 [00:00<00:00, 7672.51it/s]
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


['Write down:',
 'Save 1.',
 'a pinch of salt',
 '1, 2. (a)',
 'smart as a whip',
 'Save As',
 'Rain 3.',
 'The seat...',
 '4 (a) What does it mean to be a Christian?',
 'The mountain...',
 'Ramakrishna (cricketer)',
 'The Floss',
 'The mountain...',
 'Select the correct answer from the field below:',
 '(Great, great, great, great.)',
 'The rain...',
 '1 (a)',
 "In Krishnadevaraya's court",
 'was present.',
 'Source of water',
 'The culture']

In [25]:
!pwd

/content/indicTrans


In [26]:
!cd ..

In [27]:
!wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
!unzip m2m.zip

--2021-08-20 06:36:44--  https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 74.125.140.128, 108.177.15.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4081990185 (3.8G) [application/zip]
Saving to: ‘m2m.zip’

m2m.zip             100%[===================>]   3.80G   107MB/s    in 72s     

2021-08-20 06:37:57 (54.1 MB/s) - ‘m2m.zip’ saved [4081990185/4081990185]

Archive:  m2m.zip
   creating: m2m/
   creating: m2m/vocab/
  inflating: m2m/vocab/vocab.SRC     
  inflating: m2m/vocab/vocab.TGT     
  inflating: m2m/vocab/bpe_codes.32k.SRC_TGT  
   creating: m2m/final_bin/
  inflating: m2m/final_bin/dict.TGT.txt  
  inflating: m2m/final_bin/dict.SRC.txt  
   creating: m2m/model/
  inflating: m2m/model/checkpoint_best.pt  


In [54]:
!cd 
!cd /content/indicTrans

In [57]:
!pwd
!cp /content/indicTrans/m2m/vocab/bpe_codes.32k.SRC_TGT /content/indicTrans/m2m/vocab/bpe_codes.32k.SRC


/content/indicTrans


In [59]:
from indicTrans.inference.engine import Model
indicm2m_model = Model(expdir='/content/indicTrans/m2m')

Initializing vocab and bpe
Initializing model for translation


In [61]:
indicm2m_model.batch_translate(result, 'kn', 'hi')

100%|██████████| 21/21 [00:00<00:00, 5547.67it/s]


['कृपया इस प्रकार लिखेंः',
 'सुरक्षित रखें।',
 'बूंद-बूंद',
 '2 में से',
 'बुद्धिमान व्यक्ति',
 'इसे सहेजें',
 'बारिश',
 'सीटों की संख्या',
 '4 नंबर',
 'पर्वत',
 'राम कृष्ण',
 'पीठ',
 'पर्वतमाला',
 'सही जवाब चुनने के लिए नीचे दिए गए पते पर लिखेंः',
 '(शानदार, अप्पू, पंडित,',
 'बारिश होती है।',
 '१. संपर्क',
 'श्री कृष्णदेवराय के दरबार में',
 'वे मौजूद थे।',
 'जल स्रोत',
 'संस्कृत']